In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import igraph
import csv

# Import Data

In [ ]:
import sqlite3
conn = sqlite3.connect('C:\sqlite\eth.db')

In [ ]:
cur = conn.cursor()

## Metadata
  ____________

### Transaction count

In [ ]:
transact_count = cur.execute('''SELECT COUNT(hash) FROM transactions''').fetchall()[0][0]
print('Total transactions:', transact_count)

### Address count

In [ ]:
from_addresses = cur.execute('''SELECT from_address FROM transactions''').fetchall()
to_addresses = cur.execute('''SELECT to_address FROM transactions''').fetchall()


In [ ]:
from_addresses = [item[0] for item in from_addresses]
to_addresses = [item[0] for item in to_addresses]

In [ ]:
print('from addresses', len(from_addresses))
print('to addresses', len(to_addresses))
from_addresses[5]

### Replace null vals with contract creation

In [ ]:
to_addresses_nonull = []
for addr in to_addresses:
    if addr != '': to_addresses_nonull.append(addr)
    else: to_addresses_nonull.append('contract_creation')

In [ ]:
print('Unique from addresses:', len(set(from_addresses)))
print('Unique to addresses:', len(set(to_addresses)))

In [ ]:
all_addresses_unique = []
from_addresses_np = np.array(from_addresses)
from_addresses_unique = np.unique(from_addresses_np)


In [ ]:
len(from_addresses_unique)

In [ ]:
to_addresses_np = np.array(to_addresses_nonull)
to_addresses_unique = np.unique(to_addresses_np)
len(to_addresses_unique)

In [ ]:
all_addresses_unique = np.concatenate((from_addresses_unique, to_addresses_unique), axis=None)
all_addresses_unique = np.unique(all_addresses_unique)
print(len(all_addresses_unique))

In [ ]:
with open('unique_addresses.csv', 'w', newline='') as f:
    csvWriter=csv.writer(f)
    for addr in all_addresses_unique:
        csvWriter.writerow([addr])

In [ ]:
all_addresses_unique[0:5]

### Timeframe

In [ ]:
dates = cur.execute('''SELECT MIN(block_timestamp), MAX(block_timestamp) FROM transactions''').fetchall()
import datetime
print('Start date:', datetime.datetime.fromtimestamp(dates[0][0]))
print('End date:', datetime.datetime.fromtimestamp(dates[0][1]))
print('Days in dataset:', datetime.datetime.fromtimestamp(dates[0][1]) - datetime.datetime.fromtimestamp(dates[0][0]))

### Blocks

In [ ]:
#The beginning and ending block for the dataset
blocks = cur.execute('''SELECT MIN(block_number), MAX(block_number) FROM transactions''').fetchall()
print('First block:', blocks[0][0])
print('Last block:', blocks[0][1])
print('Difference:', blocks[0][1] - blocks[0][0], 'blocks')

## Column Statistics
____________________

### Read in cols from db

In [ ]:
value = cur.execute('''SELECT value FROM transactions''').fetchall()

In [ ]:
value = [item[0] for item in value]

In [ ]:
gas = cur.execute('''SELECT gas FROM transactions''').fetchall()

In [ ]:
gas = [item[0] for item in gas]

In [ ]:
nonce = cur.execute('''SELECT nonce FROM transactions''').fetchall()

In [ ]:
nonce = [item[0] for item in nonce]

In [ ]:
gas_price = cur.execute('''SELECT gas_price FROM transactions''').fetchall()

In [ ]:
gas_price = [item[0] for item in gas_price]

### Cols to df

In [ ]:
cols = {'value': value, 'gas': gas, 'nonce': nonce, 'gas_price': gas_price}
attr_col_df = pd.DataFrame(data=cols)

### Calc stats on cols

In [ ]:
col_stats = {'name': [], 'min': [], 'max': [], 'mean': [], 'std': [], 'var': [], 'skew': []}


In [ ]:
for col in cols.keys():
    col_stats['name'].append(col)
    col_stats['min'].append(np.min(cols[col]))
    col_stats['max'].append(np.max(cols[col]))
    col_stats['mean'].append(np.mean(cols[col]))
    col_stats['std'].append(np.std(cols[col]))
    col_stats['var'].append(np.var(cols[col]))
    col_stats['skew'].append(attr_col_df[col].skew())

In [ ]:
col_stats_df = pd.DataFrame(data=col_stats)
col_stats_df

## Handling skewness

In [ ]:
col_df_adj = pd.DataFrame()

In [ ]:
value_adj = [item+1 for item in value]
col_df_adj['value_adj'] = np.log(value_adj)
print('Skewness:', col_df_adj['value_adj'].skew())

In [ ]:
col_df_adj['gas_adj'] = np.log(gas)
print('Skewness:', col_df_adj['gas_adj'].skew())

In [ ]:
col_df_adj['nonce_adj'] = np.log(nonce)
print('Skewness:', col_df_adj['nonce_adj'].skew())

In [ ]:
col_df_adj['gas_price'] = np.log(gas_price)
print('Skewness:',col_df_adj['gas_price'].skew())

## Histograms

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(17, 8)
fig.tight_layout()
col = 0
for i in range(2):
    for j in range(2):
        ax[i][j].hist(col_df_adj[col_df_adj.columns[col]], bins=100)
        ax[i][j].set_title(col_df_adj.columns[col] +' Log transformed')
        col += 1

## Scatterplots

In [ ]:
value = col_df_adj['value_adj']
fig, ax = plt.subplots(4, 4)
fig.set_size_inches(17, 8)
fig.tight_layout()
col = 0
for i in range(4):
    ax[0][i].scatter(col_df_adj[col_df_adj.columns[col]], col_df_adj['value_adj'])
    ax[0][i].set_title(col_df_adj.columns[col] +' vs. Value')
    col += 1
col = 0
for j in range(4):
    ax[1][j].scatter(col_df_adj[col_df_adj.columns[col]], col_df_adj['gas'])
    ax[1][j].set_title(col_df_adj.columns[col] +' vs. Gas')
    col += 1
col = 0
for k in range(4):
    ax[2][k].scatter(col_df_adj[col_df_adj.columns[col]], col_df_adj['nonce_adj'])
    ax[2][k].set_title(col_df_adj.columns[col] +' vs. Nonce')
    col += 1
col = 0
for l in range(4):
    ax[3][l].scatter(col_df_adj[col_df_adj.columns[col]], col_df_adj['gas_price'])
    ax[3][l].set_title(col_df_adj.columns[col] +' vs. Gas Price')
    col += 1
plt.show()

# Build unique address dataset
___

### Build a transaction lookup table

In [ ]:
from_address_index = {}
to_address_index = {}

for address in all_addresses_unique:
    from_address_index[address] = []
    to_address_index[address] = []

In [ ]:
for index in range(len(from_addresses)):
        address = from_addresses[index]
        from_address_index[address].append(index)
for index in range(len(to_addresses)):
        address = to_addresses[index]
        to_address_index[address].append(index)

### Dataframe

In [ ]:
address_data = {'address': all_addresses_unique, 'send_count': [], 'receive_count': [], 'avg_gas_cost': [], 'total_eth_sent': [], 'total_eth_recv': [], 'max_nonce': []}


for address in all_addresses_unique:
    address_data['send_count'].append(len(from_address_index[address]))
    address_data['receive_count'].append(len(to_address_index[address]))



In [ ]:
print(len(address_data['address']))
print(len(address_data['send_count']))
print(len(address_data['receive_count']))

### Avg gas cost

In [ ]:
for address in all_addresses_unique:
    num_sends = len(from_address_index[address])
    if num_sends == 0: 
        address_data['avg_gas_cost'].append(0)
        continue
    gas_paid = 0
    for transact_index in from_address_index[address]:
        gas_paid += gas[transact_index] * gas_price[transact_index]
    avg_gas_cost = gas_paid / num_sends
    address_data['avg_gas_cost'].append(avg_gas_cost)
    
    

In [ ]:
len(address_data['avg_gas_cost'])

### Total eth sent

In [ ]:
for address in all_addresses_unique:
    num_sends = len(from_address_index[address])
    if num_sends == 0:
        address_data['total_eth_sent'].append(0)
        continue
    total_eth_sent = 0
    for transact_index in from_address_index[address]:
        total_eth_sent += value[transact_index]
    address_data['total_eth_sent'].append(total_eth_sent)

### Total eth received

In [ ]:
for address in all_addresses_unique:
    num_recv = len(to_address_index[address])
    if num_recv == 0:
        address_data['total_eth_recv'].append(0)
        continue
    total_eth_recv = 0
    for transact_index in to_address_index[address]:
        total_eth_recv += value[transact_index]
    address_data['total_eth_recv'].append(total_eth_recv)

### Max nonce

In [ ]:
for address in all_addresses_unique:
    num_sends = len(from_address_index[address])
    if num_sends == 0:
        address_data['max_nonce'].append(0)
        continue
    max_nonce = 0
    for transact_index in from_address_index[address]:
        if nonce[transact_index] > max_nonce:
            max_nonce = nonce[transact_index]
    address_data['max_nonce'].append(max_nonce)

### Create DataFrame

In [ ]:
address_df = pd.DataFrame(data=address_data)

In [ ]:
address_df.loc[0, 'address'] = 'contract_creation'

In [ ]:
address_df.head()


In [ ]:
address_df.to_csv('address_df.csv')

In [ ]:
address_df.loc[address_df['receive_count'] == address_df['receive_count'].max()]
#This address belongs to OpenSea

In [ ]:
address_df.loc[address_df['send_count'] == address_df['send_count'].max()]
#This address belongs to a miner